In [8]:
#ライブラリを読み込む
from time import sleep
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [9]:
#複数ページから情報を取得する
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13117&sc=13119&cb=10.0&ct=20.0&et=5&md=07&md=10&cn=5&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page={}'

#変数d_listに空のリストを作成する
d_list = []

In [10]:
#1~2336をループする
for i in range(1,12):
    #変数target_urlに、アクセス先のURLを格納する
    target_url = url.format(i)
   
    #target_URLへのアクセス結果を変数rに格納
    r =requests.get(target_url)
    
    #サーバーに負荷をかけないために、１秒おきにfor文を実行する
    sleep(1)

    #取得結果を解析してsoupに格納
    soup = BeautifulSoup(r.text)

    #すべての物件情報を取得する
    contents = soup.find_all("div",class_="cassetteitem")

    #各物件情報をforループで取得する
    for content in contents:
       #物件情報・部屋情報を取得する
       detail = content.find("div",class_="cassetteitem-detail")
       table = content.find("table",class_="cassetteitem_other")
       #物件情報を取得する
       title = detail.find("div",class_="cassetteitem_content-title").text 
       address = detail.find("li",class_="cassetteitem_detail-col1").text
       accsess = detail.find("li",class_="cassetteitem_detail-col2").text
       age = detail.find("li",class_="cassetteitem_detail-col3").text
       #物件情報から各部屋の情報を取得する
       tr_tags = table.find_all("tr",class_="js-cassette_link")
       for tr_tag in tr_tags:
          #部屋情報の行から欲しい情報を取得する
          floor,price,first_fee,capacity = tr_tag.find_all("td")[2:6]
          #更に細かい情報を取得する
          fee,management_fee = price.find_all("li")
          deposit,gratuity = first_fee.find_all("li")
          madori,menseki = capacity.find_all("li")
          #取得した情報を辞書に格納する
          d = {
             "title":title,
             "address":address,
             "accsess":accsess,
             "age":age,
             "floor":floor.text,
             "fee":fee.text,
             "management_fee":management_fee.text,
             "deposit":deposit.text,
             "gratuity":gratuity.text,
             "madori":madori.text,
             "menseki":menseki.text
             }
          #取得した辞書をd_listに格納する
          d_list.append(d)


In [11]:
#変数d=listを使って、データフレームを作成する
df = pd.DataFrame(d_list)
#to_csv()を使って、データフレームをCSV出力する
df.to_csv("suumo_scraping.csv",index=None,encoding="utf-8-sig")